In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns

sns.set_style("white")
plt.rcParams["figure.figsize"] = (20, 20)

import torch
import random
import pickle
import numpy as np
import pandas as pd
from tqdm import tqdm
import networkx as nx
from itertools import combinations

from fastai import *
from fastai.collab import *
from fastai.train import *

In [ ]:
with open("/mnt/efs/wikipedia/good_article_links.pkl", "rb") as fp:
    graph_dict = pickle.load(fp)

In [ ]:
G = nx.from_dict_of_lists(graph_dict)

edges = set(G.edges)
no_edges = random.sample(set(combinations(list(G.nodes), r=2)), len(edges))

In [ ]:
df_0s = pd.DataFrame(no_edges)
df_0s.columns = ["source", "target"]
df_0s["value"] = 0

In [ ]:
df_1s = pd.DataFrame(list(edges))
df_1s.columns = ["source", "target"]
df_1s["value"] = 1

In [ ]:
df = pd.concat([df_0s.sample(len(df_1s)), df_1s]).sample(frac=1)

In [ ]:
df.to_pickle("/mnt/efs/wikipedia/sample_good_article_links.pkl")

In [ ]:
databunch = CollabDataBunch.from_df(df.sample(frac=0.02))

In [ ]:
learn = collab_learner(databunch, n_factors=50, y_range=(0, 1))

In [ ]:
learn.lr_find()

In [ ]:
learn.recorder.plot()

In [ ]:
learn.fit_one_cycle(
    10,
    5e-3,
    wd=0.1,
)

In [ ]:
learn.recorder.plot_losses()

In [ ]:
df